In [1]:
from flask import Flask, request, jsonify
from threading import Thread
import json

In [2]:
app = Flask(__name__)
received_data = []

In [3]:
!pip install nest-asyncio

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Any
from pydantic import BaseModel, Field
import json
import pytz
from enum import Enum
import uuid
import re
from dateutil import parser
from dateutil.relativedelta import relativedelta
from openai import OpenAI
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

In [6]:
BASE_URL = "http://localhost:6000/v1"
MODEL_PATH = "/home/user/Models/meta-llama/Meta-Llama-3.1-8B-Instruct"

model = OpenAI(
        api_key="not-needed",
        base_url=BASE_URL,
        timeout=10.0,
    )

In [7]:
class EventModel(BaseModel):
    StartTime: str
    EndTime: str
    NumAttendees: int
    Attendees: List[str]
    Summary: str

class AttendeeModel(BaseModel):
    email: str
    events: List[EventModel] = []

class InputRequestModel(BaseModel):
    Request_id: str
    Datetime: str
    Location: str
    From: str
    Attendees: List[Dict[str, str]]
    Subject: str
    EmailContent: str

class OutputResponseModel(BaseModel):
    Request_id: str
    Datetime: str
    Location: str
    From: str
    Attendees: List[AttendeeModel]
    Subject: str
    EmailContent: str
    EventStart: str
    EventEnd: str
    Duration_mins: str
    MetaData: Dict[str, Any] = {}

class MeetingProposal(BaseModel):
    start_time: datetime
    end_time: datetime
    duration_minutes: int
    participants: List[str]
    subject: str

class NegotiationStep(BaseModel):
    step: int
    proposal: MeetingProposal
    conflicts: Dict[str, List[str]]
    resolution_strategy: str
    timestamp: datetime

In [8]:
class FlexibleDateParser:
    def __init__(self):
        self.timezone = pytz.timezone('Asia/Kolkata')
    
    def parse_flexible_datetime(self, time_text: str, reference_datetime: str) -> datetime:
        """Parse various date/time formats flexibly"""
        # **FIX**: Handle different date formats
        if 'T' in reference_datetime:
            if reference_datetime.count('-') == 2 and reference_datetime.index('-') == 2:
                # Format: "19-07-2025T12:34:55" -> "2025-07-19T12:34:55"
                parts = reference_datetime.split('T')
                date_part = parts[0]
                time_part = parts[1]
                day, month, year = date_part.split('-')
                reference_datetime = f"{year}-{month}-{day}T{time_part}"
        
        try:
            ref_dt = datetime.fromisoformat(reference_datetime.replace('T', ' '))
        except ValueError:
            # Additional fallback for problematic date formats
            try:
                from dateutil import parser as date_parser
                ref_dt = date_parser.parse(reference_datetime)
            except:
                ref_dt = datetime.now()
        
        time_text_lower = time_text.lower().strip()
        
        # Handle "tomorrow"
        if "tomorrow" in time_text_lower:
            base_date = ref_dt + timedelta(days=1)
            return self._extract_time_from_text(time_text_lower, base_date)
        
        # Handle "today"
        if "today" in time_text_lower:
            return self._extract_time_from_text(time_text_lower, ref_dt)
        
        # Handle "next week"
        if "next week" in time_text_lower:
            base_date = ref_dt + timedelta(days=7)
            return self._extract_time_from_text(time_text_lower, base_date)
        
        # Handle specific weekdays
        weekdays = {
            'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3,
            'friday': 4, 'saturday': 5, 'sunday': 6
        }
        
        for day_name, day_num in weekdays.items():
            if day_name in time_text_lower:
                if "next" in time_text_lower:
                    # Next occurrence of this weekday
                    days_ahead = (day_num - ref_dt.weekday() + 7) % 7
                    if days_ahead == 0:
                        days_ahead = 7
                else:
                    # This week or next week
                    days_ahead = (day_num - ref_dt.weekday()) % 7
                    if days_ahead == 0 and ref_dt.hour >= 17:  # If it's late, assume next week
                        days_ahead = 7
                
                target_date = ref_dt + timedelta(days=days_ahead)
                return self._extract_time_from_text(time_text_lower, target_date)
        
        # Handle specific dates
        try:
            from dateutil import parser as date_parser
            parsed_date = date_parser.parse(time_text, default=ref_dt)
            return parsed_date
        except:
            # Default fallback
            return ref_dt.replace(hour=10, minute=30, second=0, microsecond=0)
    
    def _extract_time_from_text(self, text: str, base_date: datetime) -> datetime:
        """Extract time from text and apply to base date"""
        # Look for specific times
        time_patterns = [
            (r'(\d{1,2}):(\d{2})\s*(am|pm)', lambda m: self._convert_12h_to_24h(int(m.group(1)), int(m.group(2)), m.group(3))),
            (r'(\d{1,2})\s*(am|pm)', lambda m: self._convert_12h_to_24h(int(m.group(1)), 0, m.group(2))),
            (r'(\d{1,2}):(\d{2})', lambda m: (int(m.group(1)), int(m.group(2)))),
        ]
        
        for pattern, converter in time_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                hour, minute = converter(match)
                return base_date.replace(hour=hour, minute=minute, second=0, microsecond=0)
        
        # Handle general time references
        if "morning" in text:
            return base_date.replace(hour=9, minute=0, second=0, microsecond=0)
        elif "afternoon" in text:
            return base_date.replace(hour=14, minute=0, second=0, microsecond=0)
        elif "evening" in text:
            return base_date.replace(hour=17, minute=0, second=0, microsecond=0)
        else:
            # Default to 10:30 AM
            return base_date.replace(hour=10, minute=30, second=0, microsecond=0)
    
    def _convert_12h_to_24h(self, hour: int, minute: int, period: str) -> tuple:
        """Convert 12-hour format to 24-hour format"""
        if period.lower() == 'pm' and hour != 12:
            hour += 12
        elif period.lower() == 'am' and hour == 12:
            hour = 0
        return hour, minute


In [9]:
class EmailParsingResult(BaseModel):
    duration_minutes: int = Field(description="Meeting duration in minutes")
    time_preferences: str = Field(description="Raw time preference from email")
    parsed_datetime: str = Field(description="Parsed datetime in ISO format")
    subject: str = Field(description="Meeting subject/title")
    constraints: str = Field(description="Any specific constraints or preferences")

class EmailParsingAgent:
    def __init__(self):
        self.client = model  
        self.model_path = MODEL_PATH
        self.date_parser = FlexibleDateParser()
    
    def _get_system_prompt(self):
        return """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an intelligent meeting scheduling agent. Parse emails and extract meeting details.

CRITICAL: Extract the EXACT time mentioned in the email, not default times.

Examples:

Email: "Let's meet Monday at 9:00 AM to discuss and resolve this issue."
Reference: "19-07-2025T12:34:55"
Output: {"duration_minutes": 30, "time_preferences": "Monday at 9:00 AM", "parsed_datetime": "2025-07-21T09:00:00", "subject": "CEO Meet - Urgent", "constraints": "specific time mentioned"}

Email: "Hi Team. Let's meet on Tuesday at 11:00 A.M and discuss about our on-going Projects."
Reference: "19-07-2025T12:34:55"  
Output: {"duration_minutes": 30, "time_preferences": "Tuesday at 11:00 A.M", "parsed_datetime": "2025-07-22T11:00:00", "subject": "Project Status", "constraints": "specific time mentioned"}

Email: "Can we schedule a meeting next Monday morning for an hour to review the budget?"
Reference: "19-07-2025T12:34:55"
Output: {"duration_minutes": 60, "time_preferences": "next Monday morning", "parsed_datetime": "2025-07-28T09:00:00", "subject": "budget review", "constraints": "morning preference"}

Time parsing rules:
- "Monday at 9:00 AM" = Monday 09:00 (USE EXACT TIME)
- "Tuesday at 11:00 A.M" = Tuesday 11:00 (USE EXACT TIME)  
- "tomorrow at 2 PM" = next day 14:00 (USE EXACT TIME)
- "Thursday morning" = Thursday 09:00 (morning default)
- "next Monday" = Monday of next week
- Default duration: 30 minutes if not specified
- ALWAYS use the exact time if specified in email

Always return valid JSON only.<|eot_id|>"""
    
    def parse_email(self, email_content: str, attendees: List[str], reference_datetime: str) -> EmailParsingResult:
        """Parse email using direct OpenAI client calls"""
        
        user_prompt = f"""Parse this email and extract meeting details:

Email: "{email_content}"
Reference Date: "{reference_datetime}"
Known Attendees: {attendees}

IMPORTANT: If the email says "Monday at 9:00 AM", the parsed_datetime should be Monday at 09:00, NOT 10:30.
Extract the EXACT time mentioned, don't use defaults unless no time is specified.

Return JSON with: duration_minutes, time_preferences, parsed_datetime, subject, constraints"""
        
        messages = [
            {"role": "system", "content": self._get_system_prompt()},
            {"role": "user", "content": user_prompt}
        ]
        
        try:
            # Direct OpenAI client call with safe parameters
            response = self.client.chat.completions.create(
                model=self.model_path,
                messages=messages,
                temperature=0.1,
                max_tokens=500,
                top_p=0.9
            )
            
            content = response.choices[0].message.content.strip()
            
            # Try to parse JSON response
            try:
                parsed_data = json.loads(content)
            except json.JSONDecodeError:
                # Try to extract JSON from content
                json_match = re.search(r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}', content, re.DOTALL)
                if json_match:
                    try:
                        parsed_data = json.loads(json_match.group())
                    except:
                        return self._fallback_parse(email_content, reference_datetime)
                else:
                    return self._fallback_parse(email_content, reference_datetime)
            
            # **CRITICAL FIX**: Validate and correct the parsed datetime using enhanced parser
            try:
                corrected_datetime = self.date_parser.parse_flexible_datetime(
                    parsed_data.get('time_preferences', email_content), 
                    reference_datetime
                )
                parsed_data['parsed_datetime'] = corrected_datetime.isoformat()
            except Exception as e:
                # Use enhanced fallback
                corrected_datetime = self._enhanced_fallback_datetime(email_content, reference_datetime)
                parsed_data['parsed_datetime'] = corrected_datetime.isoformat()
            
            # Create EmailParsingResult with validated data
            return EmailParsingResult(
                duration_minutes=parsed_data.get('duration_minutes', 30),
                time_preferences=parsed_data.get('time_preferences', email_content),
                parsed_datetime=parsed_data.get('parsed_datetime', ''),
                subject=parsed_data.get('subject', 'Meeting'),
                constraints=parsed_data.get('constraints', '')
            )
            
        except Exception as e:
            print(f"Email parsing error: {e}")
            return self._fallback_parse(email_content, reference_datetime)
    
    def _enhanced_fallback_datetime(self, email_content: str, reference_datetime: str) -> datetime:
        """Enhanced fallback that properly extracts time from email"""
        # Fix reference datetime format first
        if 'T' in reference_datetime and reference_datetime.count('-') == 2 and reference_datetime.index('-') == 2:
            parts = reference_datetime.split('T')
            date_part = parts[0]
            time_part = parts[1]
            day, month, year = date_part.split('-')
            reference_datetime = f"{year}-{month}-{day}T{time_part}"
        
        ref_dt = datetime.fromisoformat(reference_datetime.replace('T', ' '))
        
        # **ENHANCED TIME EXTRACTION**
        email_lower = email_content.lower()
        
        # Look for specific times first
        time_patterns = [
            (r'(\d{1,2}):(\d{2})\s*(a\.?m\.?|p\.?m\.?)', lambda m: self._convert_12h_to_24h(int(m.group(1)), int(m.group(2)), m.group(3))),
            (r'(\d{1,2})\s*(a\.?m\.?|p\.?m\.?)', lambda m: self._convert_12h_to_24h(int(m.group(1)), 0, m.group(2))),
        ]
        
        extracted_time = None
        for pattern, converter in time_patterns:
            match = re.search(pattern, email_lower)
            if match:
                hour, minute = converter(match)
                extracted_time = (hour, minute)
                break
        
        # Look for weekdays
        weekdays = {
            'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3,
            'friday': 4, 'saturday': 5, 'sunday': 6
        }
        
        target_date = ref_dt
        for day_name, day_num in weekdays.items():
            if day_name in email_lower:
                days_ahead = (day_num - ref_dt.weekday()) % 7
                if days_ahead == 0 and ref_dt.hour >= 17:
                    days_ahead = 7
                target_date = ref_dt + timedelta(days=days_ahead)
                break
        
        # Apply extracted time or use defaults
        if extracted_time:
            hour, minute = extracted_time
            return target_date.replace(hour=hour, minute=minute, second=0, microsecond=0)
        else:
            # Default to 10:30 AM if no specific time found
            return target_date.replace(hour=10, minute=30, second=0, microsecond=0)
    
    def _convert_12h_to_24h(self, hour: int, minute: int, period: str) -> tuple:
        """Convert 12-hour format to 24-hour format"""
        period = period.lower().replace('.', '')
        if period == 'pm' and hour != 12:
            hour += 12
        elif period == 'am' and hour == 12:
            hour = 0
        return hour, minute
    
    def _fallback_parse(self, email_content: str, reference_datetime: str) -> EmailParsingResult:
        """Enhanced regex-based fallback parsing"""
        duration = 30
        subject = "Meeting"
        constraints = "fallback parsing used"
        
        # Extract duration
        duration_patterns = [
            r'(\d+)\s*minutes?',
            r'(\d+)\s*mins?',
            r'for\s+(\d+)',
            r'(\d+)\s*hour'
        ]
        
        for pattern in duration_patterns:
            match = re.search(pattern, email_content, re.IGNORECASE)
            if match:
                found_duration = int(match.group(1))
                if 'hour' in pattern:
                    found_duration *= 60
                duration = found_duration
                break
        
        # Extract subject
        if 'ceo' in email_content.lower() or 'urgent' in email_content.lower():
            subject = "CEO Meet - Urgent"
        elif 'project' in email_content.lower():
            subject = "Project Status"
        elif 'status' in email_content.lower():
            subject = "Status update"
        
        # Use enhanced datetime parsing
        try:
            parsed_dt = self._enhanced_fallback_datetime(email_content, reference_datetime)
            parsed_datetime = parsed_dt.isoformat()
        except Exception as e:
            print(f"Final fallback datetime error: {e}")
            # Ultimate fallback
            try:
                ref_dt = datetime.now()
                parsed_datetime = ref_dt.replace(hour=10, minute=30).isoformat()
            except:
                parsed_datetime = "2025-07-21T09:00:00"
        
        return EmailParsingResult(
            duration_minutes=duration,
            time_preferences=email_content,
            parsed_datetime=parsed_datetime,
            subject=subject,
            constraints=constraints
        )


In [10]:
def retrieve_calendar_events(user: str, start_date: str, end_date: str) -> List[Dict]:
    """Tool to retrieve calendar events for a user"""
    try:
        events_list = []
        token_path = f"Keys/{user.split('@')[0]}.token"
        
        from google.auth.transport.requests import Request
        from google.oauth2.credentials import Credentials
        from googleapiclient.discovery import build
        
        user_creds = Credentials.from_authorized_user_file(token_path)
        calendar_service = build("calendar", "v3", credentials=user_creds)
        
        events_result = calendar_service.events().list(
            calendarId='primary', 
            timeMin=start_date,
            timeMax=end_date,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        
        events = events_result.get('items', [])
        
        for event in events:
            attendee_list = []
            try:
                for attendee in event.get("attendees", []):
                    attendee_list.append(attendee['email'])
            except:
                attendee_list.append("SELF")
            
            start_time = event["start"].get("dateTime", event["start"].get("date"))
            end_time = event["end"].get("dateTime", event["end"].get("date"))
            
            events_list.append({
                "StartTime": start_time,
                "EndTime": end_time,
                "NumAttendees": len(set(attendee_list)),
                "Attendees": list(set(attendee_list)),
                "Summary": event.get("summary", "No Title")
            })
        
        return events_list
    except Exception as e:
        return [{"error": f"Failed to retrieve calendar for {user}: {str(e)}"}]

def find_conflicts(proposal_start: str, proposal_end: str, user_events: List[Dict]) -> List[str]:
    """Enhanced conflict detection with better timezone and overlap logic"""
    conflicts = []
    timezone = pytz.timezone('Asia/Kolkata')
    
    try:
        # Parse proposal times
        prop_start = datetime.fromisoformat(proposal_start.replace('Z', '+00:00'))
        prop_end = datetime.fromisoformat(proposal_end.replace('Z', '+00:00'))
        
        # Ensure timezone awareness
        if prop_start.tzinfo is None:
            prop_start = timezone.localize(prop_start)
        if prop_end.tzinfo is None:
            prop_end = timezone.localize(prop_end)
        
        
        for event in user_events:
            if "error" in event:
                continue
                
            try:
                event_start_str = event["StartTime"]
                event_end_str = event["EndTime"]
                
                
                # Parse event times with timezone handling
                if 'T' in event_start_str:
                    event_start = datetime.fromisoformat(event_start_str.replace('Z', '+00:00'))
                    event_end = datetime.fromisoformat(event_end_str.replace('Z', '+00:00'))
                else:
                    # All-day events
                    event_start = datetime.fromisoformat(f"{event_start_str}T00:00:00")
                    event_end = datetime.fromisoformat(f"{event_end_str}T23:59:59")
                
                # Ensure timezone awareness for events
                if event_start.tzinfo is None:
                    event_start = timezone.localize(event_start)
                if event_end.tzinfo is None:
                    event_end = timezone.localize(event_end)
                
                # **FIXED OVERLAP LOGIC**: Check if times overlap
                # Two time ranges overlap if: start1 < end2 AND start2 < end1
                if prop_start < event_end and event_start < prop_end:
                    conflict_msg = f"Conflicts with: {event.get('Summary', 'No Title')} ({event_start_str} - {event_end_str})"
                    conflicts.append(conflict_msg)
                else:
                    print(f"No conflict with: {event.get('Summary', 'No Title')}")
                    
            except Exception as e:
                print(f"Error processing event {event}: {e}")
                continue
        
        return conflicts
        
    except Exception as e:
        print(f"Error in find_conflicts: {e}")
        return [f"Error checking conflicts: {str(e)}"]



In [11]:
class NegotiationInput(BaseModel):
    current_start: str = Field(description="Current proposed start time")
    current_end: str = Field(description="Current proposed end time") 
    duration_minutes: int = Field(description="Meeting duration in minutes")
    conflicts: Dict[str, List[str]] = Field(description="Conflicts for each user")
    step: int = Field(description="Negotiation step number")
    participants: List[str] = Field(description="Meeting participants")

class NegotiationResult(BaseModel):
    new_start_time: str = Field(description="New proposed start time in ISO format")
    new_end_time: str = Field(description="New proposed end time in ISO format")
    reasoning: str = Field(description="Explanation for the time change")
    strategy: str = Field(description="Strategy used for negotiation")

class NegotiationAgent:
    def __init__(self):
        self.client = model
        self.model_path = MODEL_PATH
    
    def _get_system_prompt(self):
        return """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an intelligent scheduling negotiation agent. Find alternative meeting times when conflicts exist.

CRITICAL RULES:
1. If conflicts span entire day (like workshops, training, all-day events), MOVE TO NEXT DAY
2. If multiple people have conflicts on same day, MOVE TO NEXT DAY  
3. Only try same-day alternatives if conflicts are short (1-2 hours)
4. Maintain the same duration and time of day when moving to next day

Conflict Analysis:
- Workshop/Training events (6+ hours) = NEXT DAY required
- Multiple conflicts on same day = NEXT DAY required  
- Single short conflict = Same day shift possible

Strategies:
- "next_day_same_time": Move to next day, keep same time (PREFERRED for workshops)
- "next_available_day": Skip to next completely free day
- "same_day_shift": Only for short conflicts (1-2 hours)
- "avoid_weekends": Skip Saturday/Sunday

Examples:
- Workshop 9AM-5PM Tuesday → Wednesday 10:30 AM (next_day_same_time)
- Multiple conflicts Tuesday → Wednesday same time (next_day_same_time)  
- Single 1-hour conflict → Same day +2 hours (same_day_shift)

Always return valid JSON with: new_start_time, new_end_time, reasoning, strategy<|eot_id|>"""
    
    def negotiate_time_slot(self, 
                          current_proposal: MeetingProposal, 
                          conflicts: Dict[str, List[str]], 
                          step: int) -> MeetingProposal:
        """Enhanced negotiation with smart day-change logic"""
        
        # **STEP 1: Analyze conflict patterns**
        conflict_analysis = self._analyze_conflict_patterns(conflicts)
        
        # **STEP 2: Determine if we need to move to next day**
        if self._should_move_to_next_day(conflicts, conflict_analysis, step):
            return self._move_to_next_day(current_proposal, step)
        
        # **STEP 3: Try AI negotiation first**
        ai_proposal = self._try_ai_negotiation(current_proposal, conflicts, step)
        
        # **STEP 4: Validate AI proposal doesn't stay on same problematic day**
        if self._is_same_problematic_day(current_proposal, ai_proposal, conflicts):
            return self._move_to_next_day(current_proposal, step)
        
        return ai_proposal
    
    def _analyze_conflict_patterns(self, conflicts: Dict[str, List[str]]) -> dict:
        """Analyze conflict patterns to determine best strategy"""
        analysis = {
            "total_conflicts": 0,
            "users_with_conflicts": 0,
            "has_workshop": False,
            "has_all_day": False,
            "has_long_events": False
        }
        
        for user, user_conflicts in conflicts.items():
            if user_conflicts:
                analysis["users_with_conflicts"] += 1
                analysis["total_conflicts"] += len(user_conflicts)
                
                for conflict in user_conflicts:
                    conflict_lower = conflict.lower()
                    if "workshop" in conflict_lower or "training" in conflict_lower:
                        analysis["has_workshop"] = True
                    if "09:00:00" in conflict and "17:00:00" in conflict:
                        analysis["has_all_day"] = True
                    if any(keyword in conflict_lower for keyword in ["workshop", "training", "conference", "seminar"]):
                        analysis["has_long_events"] = True
        
        return analysis
    
    def _should_move_to_next_day(self, conflicts: Dict[str, List[str]], analysis: dict, step: int) -> bool:
        """Determine if we should move to next day"""
        # Force next day if:
        # 1. Workshop/training conflicts detected
        if analysis["has_workshop"] or analysis["has_all_day"] or analysis["has_long_events"]:
            return True
        
        # 2. Multiple people have conflicts (medium/high severity)
        if analysis["users_with_conflicts"] >= 2:
            print(f"Multiple users ({analysis['users_with_conflicts']}) have conflicts - moving to next day")
            return True
        
        # 3. After 2 failed attempts on same day
        if step >= 3:
            return True
        
        return False
    
    def _move_to_next_day(self, current_proposal: MeetingProposal, step: int) -> MeetingProposal:
        """Move meeting to next available day with same time"""
        original_time = current_proposal.start_time.time()
        current_date = current_proposal.start_time.date()
        
        # Calculate days to add based on step
        if step <= 2:
            days_to_add = 1  # Next day
        elif step == 3:
            days_to_add = 2  # Day after tomorrow
        else:
            days_to_add = step  # Keep moving forward
        
        new_date = current_date + timedelta(days=days_to_add)
        
        # Skip weekends
        while new_date.weekday() >= 5:  # Saturday = 5, Sunday = 6
            new_date += timedelta(days=1)
        
        # Create new datetime with same time
        new_start = datetime.combine(new_date, original_time)
        new_end = new_start + timedelta(minutes=current_proposal.duration_minutes)
        
        print(f"Moving from {current_proposal.start_time.date()} to {new_date} (same time: {original_time})")
        
        return MeetingProposal(
            start_time=new_start,
            end_time=new_end,
            duration_minutes=current_proposal.duration_minutes,
            participants=current_proposal.participants,
            subject=current_proposal.subject
        )
    
    def _try_ai_negotiation(self, current_proposal: MeetingProposal, conflicts: Dict[str, List[str]], step: int) -> MeetingProposal:
        """Try AI-based negotiation"""
        conflict_summary = "\n".join([
            f"{user}: {', '.join(user_conflicts)}" 
            for user, user_conflicts in conflicts.items() if user_conflicts
        ])
        
        user_prompt = f"""Current proposal has conflicts. Find alternative time:

Current: {current_proposal.start_time} - {current_proposal.end_time}
Duration: {current_proposal.duration_minutes} minutes

Conflicts:
{conflict_summary}

Step: {step}

IMPORTANT: If conflicts mention "Workshop", "Training", or span 6+ hours, you MUST move to next day with same time.

Return JSON with: new_start_time (ISO format), new_end_time (ISO format), reasoning, strategy"""
        
        messages = [
            {"role": "system", "content": self._get_system_prompt()},
            {"role": "user", "content": user_prompt}
        ]
        
        try:
            response = self.client.chat.completions.create(
                model=self.model_path,
                messages=messages,
                temperature=0.2,  # Lower temperature for more consistent logic
                max_tokens=400,
                top_p=0.9
            )
            
            content = response.choices[0].message.content.strip()
            
            # Parse JSON response
            try:
                result = json.loads(content)
            except json.JSONDecodeError:
                json_match = re.search(r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}', content, re.DOTALL)
                if json_match:
                    try:
                        result = json.loads(json_match.group())
                    except:
                        return self._move_to_next_day(current_proposal, step)
                else:
                    return self._move_to_next_day(current_proposal, step)
            
            # Create new proposal
            return MeetingProposal(
                start_time=datetime.fromisoformat(result.get('new_start_time')),
                end_time=datetime.fromisoformat(result.get('new_end_time')),
                duration_minutes=current_proposal.duration_minutes,
                participants=current_proposal.participants,
                subject=current_proposal.subject
            )
            
        except Exception as e:
            print(f"AI negotiation error: {e}")
            return self._move_to_next_day(current_proposal, step)
    
    def _is_same_problematic_day(self, original: MeetingProposal, new: MeetingProposal, conflicts: Dict[str, List[str]]) -> bool:
        """Check if AI suggested same day that has conflicts"""
        if original.start_time.date() == new.start_time.date():
            # Same day - check if conflicts still exist
            conflict_count = sum(len(user_conflicts) for user_conflicts in conflicts.values())
            if conflict_count > 0:
                return True
        return False



In [12]:
class CalendarAnalysisInput(BaseModel):
    all_calendars: Dict[str, List[Dict]] = Field(description="All participants' calendar events")
    proposal_start: str = Field(description="Proposed meeting start time")
    proposal_end: str = Field(description="Proposed meeting end time")
    participants: List[str] = Field(description="Meeting participants")

class CalendarAnalysisResult(BaseModel):
    conflicts_found: Dict[str, List[str]] = Field(description="Conflicts for each participant")
    availability_status: Dict[str, bool] = Field(description="Availability status for each participant")
    recommendations: List[str] = Field(description="Recommendations for scheduling")
    conflict_severity: str = Field(description="Overall conflict severity: low, medium, high")


class CalendarAnalysisAgent:
    def __init__(self):
        self.client = model
        self.model_path = MODEL_PATH
    
    def _get_system_prompt(self):
        return """You are a calendar analysis agent. Analyze scheduling conflicts and provide recommendations.

IMPORTANT: You must detect conflicts by comparing the proposed meeting time with existing calendar events.

Analyze:
1. Individual conflicts for each participant  
2. Overall availability status (false if ANY conflicts exist)
3. Conflict severity based on number of people with conflicts
4. Smart recommendations for resolution

Conflict Detection Rules:
- Two events conflict if their time ranges overlap
- Event A (start1-end1) conflicts with Event B (start2-end2) if: start1 < end2 AND start2 < end1
- All-day events conflict with any meeting on that day
- Workshop/training events typically block the entire day

Severity Levels:
- "none": No conflicts found
- "low": 1 person has conflicts  
- "medium": 2 people have conflicts
- "high": 3+ people have conflicts

Always return valid JSON with: conflicts_found, availability_status, recommendations, conflict_severity"""
    
    def analyze_conflicts(self, 
                        all_calendars: Dict[str, List[EventModel]], 
                        proposal: MeetingProposal) -> CalendarAnalysisResult:
        """Enhanced conflict analysis with manual verification"""
        
        # **STEP 1: Manual conflict detection first**
        conflicts_found = {}
        availability_status = {}
        
        proposal_start_str = proposal.start_time.isoformat()
        proposal_end_str = proposal.end_time.isoformat()
        
        for user, events in all_calendars.items():
            user_events_dict = [event.model_dump() for event in events]
            
            # Use the fixed find_conflicts function
            user_conflicts = find_conflicts(
                proposal_start_str,
                proposal_end_str, 
                user_events_dict
            )
            
            conflicts_found[user] = user_conflicts
            availability_status[user] = len(user_conflicts) == 0
        
        
        # **STEP 2: Generate recommendations based on conflicts**
        total_conflicts = sum(len(conflicts) for conflicts in conflicts_found.values())
        conflict_users = [user for user, user_conflicts in conflicts_found.items() if user_conflicts]
        
        recommendations = []
        if total_conflicts == 0:
            recommendations.append("Current time works for everyone")
            severity = "none"
        elif len(conflict_users) == 1:
            recommendations.append(f"Move to avoid {conflict_users[0]}'s conflict")
            recommendations.append("Try 2-3 hours later or next day")
            severity = "low"
        elif len(conflict_users) == 2:
            recommendations.append("Multiple conflicts detected - consider alternative day")
            recommendations.append("Try next available day with same time")
            severity = "medium"
        else:
            recommendations.append("Major rescheduling needed - most participants have conflicts")
            recommendations.append("Consider completely different time slot")
            severity = "high"
        
        # **STEP 3: Create result with manual analysis**
        result = CalendarAnalysisResult(
            conflicts_found=conflicts_found,
            availability_status=availability_status,
            recommendations=recommendations,
            conflict_severity=severity
        )
        
        return result
    
    def _fallback_analysis(self, all_calendars: Dict[str, List[EventModel]], proposal: MeetingProposal) -> CalendarAnalysisResult:
        """This should now rarely be called due to manual analysis above"""
        return self.analyze_conflicts(all_calendars, proposal)



In [13]:
class NegotiationLanguageAgent:
    def __init__(self):
        self.client = model
        self.model_path = MODEL_PATH
    
    def generate_polite_negotiation(self, step_data: dict) -> str:
        """Generate human-like negotiation statements"""
        
        # Extract data from step_data
        step = step_data.get('step', 1)
        conflicts = step_data.get('conflicts', {})
        recommendations = step_data.get('recommendations', [])
        original_time = step_data.get('original_time')
        new_time = step_data.get('new_time')
        conflict_severity = step_data.get('conflict_severity', 'low')
        participants = step_data.get('participants', [])
        
        # Create context for the negotiation
        conflict_summary = self._summarize_conflicts(conflicts)
        time_context = self._format_time_context(original_time, new_time)
        
        system_prompt = """You are a professional meeting coordinator who writes polite, human-like negotiation emails. 
        
Generate natural, courteous negotiation statements that sound like they could be from a real person coordinating meetings via email.

Style Guidelines:
- Use polite, professional language
- Be empathetic to scheduling conflicts
- Offer clear alternatives
- Sound conversational but professional
- Include specific reasoning
- Be concise but warm

Examples:
- "I see there's a conflict with your 2 PM call. Would 4 PM work better for everyone?"
- "It looks like Thursday afternoon is quite busy. How about we try Friday morning at 10:30 AM instead?"
- "I understand everyone has a packed schedule. Let me suggest moving this to Tuesday at 3 PM to avoid the conflicts."
- "Thanks for your patience with the scheduling. I found a slot that should work for everyone: Wednesday at 11 AM."

Avoid:
- Robotic or overly formal language
- Technical jargon
- Long explanations
- Multiple options in one message"""

        user_prompt = f"""Generate a polite negotiation statement for this meeting scheduling situation:

Step: {step}
Conflict Severity: {conflict_severity}
{conflict_summary}
{time_context}
Recommendations: {', '.join(recommendations) if recommendations else 'None'}
Participants: {len(participants)} people

Write a natural, human-like negotiation statement that a meeting coordinator would send via email."""

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        
        try:
            response = self.client.chat.completions.create(
                model=self.model_path,
                messages=messages,
                temperature=0.7,  # Higher temperature for more natural variation
                max_tokens=200,
                top_p=0.9
            )
            
            content = response.choices[0].message.content.strip()
            
            # Clean up the response - remove quotes if present
            content = content.strip('"').strip("'")
            
            return content
            
        except Exception as e:
            print(f"Negotiation language generation error: {e}")
            return self._fallback_negotiation_statement(step_data)
    
    def _summarize_conflicts(self, conflicts: dict) -> str:
        """Summarize conflicts in a human-readable format"""
        if not conflicts or not any(conflicts.values()):
            return "No conflicts found."
        
        conflict_users = [user for user, user_conflicts in conflicts.items() if user_conflicts]
        
        if len(conflict_users) == 1:
            return f"Conflicts detected: {conflict_users[0]} has a scheduling conflict."
        elif len(conflict_users) == 2:
            return f"Conflicts detected: {conflict_users[0]} and {conflict_users[1]} have scheduling conflicts."
        else:
            return f"Conflicts detected: {len(conflict_users)} participants have scheduling conflicts."
    
    def _format_time_context(self, original_time, new_time) -> str:
        """Format time context for the negotiation"""
        if not original_time or not new_time:
            return ""
        
        if isinstance(original_time, str):
            original_time = datetime.fromisoformat(original_time.replace('Z', '+00:00'))
        if isinstance(new_time, str):
            new_time = datetime.fromisoformat(new_time.replace('Z', '+00:00'))
        
        original_formatted = original_time.strftime('%A at %I:%M %p')
        new_formatted = new_time.strftime('%A at %I:%M %p')
        
        return f"Original time: {original_formatted}\nProposed new time: {new_formatted}"
    
    def _fallback_negotiation_statement(self, step_data: dict) -> str:
        """Fallback negotiation statements when AI fails"""
        step = step_data.get('step', 1)
        conflicts = step_data.get('conflicts', {})
        
        conflict_count = sum(1 for user_conflicts in conflicts.values() if user_conflicts)
        
        fallback_statements = {
            1: {
                0: "Great! The proposed time works for everyone. Looking forward to our meeting.",
                1: "I see there's a small scheduling conflict. Let me suggest an alternative time that works better.",
                2: "I notice a couple of conflicts with the proposed time. How about we try a different slot?",
                3: "It looks like several people have conflicts. Let me find a time that works for everyone."
            },
            2: {
                0: "Perfect! The adjusted time should work for everyone.",
                1: "Thanks for your patience. I found an alternative that should resolve the conflict.",
                2: "I've identified another option that should work better for everyone's schedule.",
                3: "Let me suggest another time slot that avoids these conflicts."
            },
            3: {
                0: "Excellent! This time slot looks clear for everyone.",
                1: "I believe this new time should work much better.",
                2: "Here's another option that should accommodate everyone's availability.",
                3: "After reviewing everyone's calendars, this time should be conflict-free."
            }
        }
        
        step_key = min(step, 3)
        conflict_key = min(conflict_count, 3)
        
        return fallback_statements.get(step_key, fallback_statements[1]).get(conflict_key, 
            "Thank you for your patience as we work to find a time that works for everyone.")


In [14]:
class SchedulingOrchestrator:
    def __init__(self):
        self.email_agent = EmailParsingAgent()
        self.negotiation_agent = NegotiationAgent()
        self.calendar_analysis_agent = CalendarAnalysisAgent() 
        self.timezone = pytz.timezone('Asia/Kolkata')
        self.negotiation_language_agent = NegotiationLanguageAgent()
    
    def schedule_meeting(self, input_data: InputRequestModel) -> OutputResponseModel:
        """Enhanced orchestration method with calendar analysis"""
        metadata = {
            "negotiation_steps": [],
            "processing_log": [],
            "conflicts_resolved": [],
            "calendar_analysis": [],
            "negotiation_statements": [],
            "approach": "direct_openai_client_with_analysis"
        }
        
        # Step 1: Parse email content
        all_attendees = [input_data.From] + [att["email"] for att in input_data.Attendees]
        parsed_email = self.email_agent.parse_email(
            input_data.EmailContent, 
            all_attendees, 
            input_data.Datetime
        )
        
        metadata["processing_log"].append({
            "step": "email_parsing",
            "result": parsed_email.model_dump(),
            "timestamp": datetime.now().isoformat()
        })
        
        # Step 2: Create initial proposal
        meeting_start = datetime.fromisoformat(parsed_email.parsed_datetime)
        meeting_end = meeting_start + timedelta(minutes=parsed_email.duration_minutes)
        
        initial_proposal = MeetingProposal(
            start_time=meeting_start,
            end_time=meeting_end,
            duration_minutes=parsed_email.duration_minutes,
            participants=all_attendees,
            subject=input_data.Subject
        )
        
        # Step 3: Retrieve all calendars
        search_start = meeting_start.strftime('%Y-%m-%dT00:00:00+05:30')
        search_end = meeting_start.strftime('%Y-%m-%dT23:59:59+05:30')
        
        all_calendars = {}
        for attendee in all_attendees:
            calendar_events = retrieve_calendar_events(attendee, search_start, search_end)
            all_calendars[attendee] = [EventModel(**event) for event in calendar_events if "error" not in event]
        
        # Step 4: Enhanced negotiation loop with calendar analysis
        current_proposal = initial_proposal
        max_rounds = 5
        
        for round_num in range(max_rounds):
            # **NEW**: Perform detailed calendar analysis
            analysis_result = self.calendar_analysis_agent.analyze_conflicts(
                all_calendars, current_proposal
            )
            
            metadata["calendar_analysis"].append({
                "round": round_num + 1,
                "analysis": analysis_result.model_dump(),
                "timestamp": datetime.now().isoformat()
            })
            
            # Check for conflicts using analysis results
            conflicts = analysis_result.conflicts_found
            availability = analysis_result.availability_status
            
            # Record negotiation step with enhanced data
            metadata["negotiation_steps"].append({
                "step": round_num + 1,
                "proposal": current_proposal.model_dump(),
                "conflicts": conflicts,
                "availability_status": availability,
                "recommendations": analysis_result.recommendations,
                "conflict_severity": analysis_result.conflict_severity,
                "resolution_strategy": "conflict_detection" if any(conflicts.values()) else "no_conflicts",
                "timestamp": datetime.now().isoformat()
            })

            negotiation_statement = self.negotiation_language_agent.generate_polite_negotiation({
                "step": round_num + 1,
                "conflicts": conflicts,
                "recommendations": analysis_result.recommendations,
                "original_time": initial_proposal.start_time,
                "new_time": current_proposal.start_time
            })
            
            metadata["negotiation_statements"].append({
                "step": round_num + 1,
                "statement": negotiation_statement,
                "timestamp": datetime.now().isoformat()
            })
            
            # If no conflicts, we're done
            if not any(conflicts.values()):
                metadata["processing_log"].append({
                    "step": "scheduling_complete",
                    "final_proposal": current_proposal.model_dump(),
                    "final_analysis": analysis_result.model_dump(),
                    "rounds_needed": round_num + 1
                })
                break
            
            # Negotiate new time using AI recommendations
            if round_num < max_rounds - 1:
                # **ENHANCED**: Pass analysis recommendations to negotiation
                current_proposal = self.negotiation_agent.negotiate_time_slot(
                    current_proposal, conflicts, round_num + 1
                )
                metadata["conflicts_resolved"].append({
                    "round": round_num + 1,
                    "conflicts": conflicts,
                    "recommendations_used": analysis_result.recommendations,
                    "new_proposal": current_proposal.model_dump()
                })
        
        # Step 5: Build final response with analysis insights
        final_attendees = []
        for attendee_email in all_attendees:
            user_events = all_calendars[attendee_email].copy()
            
            filtered_events = []
            for event in user_events:
                if event.Summary != "Off Hours":
                    filtered_events.append(event)
            num_attendees = len(all_attendees) if len(all_attendees) > 0 else 1
            attendees_list = all_attendees if all_attendees else ["SELF"]
            
            filtered_events.append(EventModel(
                StartTime=current_proposal.start_time.strftime('%Y-%m-%dT%H:%M:%S+05:30'),
                EndTime=current_proposal.end_time.strftime('%Y-%m-%dT%H:%M:%S+05:30'),
                NumAttendees=len(all_attendees),
                Attendees=all_attendees,
                Summary=input_data.Subject,
            ))
            
            final_attendees.append(AttendeeModel(
                email=attendee_email,
                events=filtered_events 
            ))
        
        return OutputResponseModel(
            Request_id=input_data.Request_id,
            Datetime=input_data.Datetime,
            Location=input_data.Location,
            From=input_data.From,
            Attendees=final_attendees,
            Subject=input_data.Subject,
            EmailContent=input_data.EmailContent,
            EventStart=current_proposal.start_time.strftime('%Y-%m-%dT%H:%M:%S+05:30'),
            EventEnd=current_proposal.end_time.strftime('%Y-%m-%dT%H:%M:%S+05:30'),
            Duration_mins=str(current_proposal.duration_minutes),
            MetaData=metadata
        )


In [15]:
orchestrator = SchedulingOrchestrator()

In [16]:
def your_meeting_assistant(data): 
    input_data = InputRequestModel(**data)
    result = orchestrator.schedule_meeting(input_data)
    return json.loads(result.model_dump_json())

In [17]:
@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\n Received: {json.dumps(data, indent=2)}")
    new_data = your_meeting_assistant(data)  # Your AI Meeting Assistant Function Call
    received_data.append(data)
    print(f"\n\n\n Sending:\n {json.dumps(new_data, indent=2)}")
    return jsonify(new_data)

def run_flask():
    app.run(host='0.0.0.0', port=5000)

In [18]:
# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://129.212.191.68:5000
Press CTRL+C to quit



 Received: {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "Subject": "CEO position status Update",
  "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of new CEO position."
}
No conflict with: Off Hours
No conflict with: Off Hours
No conflict with: Off Hours
No conflict with: Off Hours
No conflict with: Off Hours
No conflict with: Off Hours
Multiple users (3) have conflicts - moving to next day
Moving from 2025-07-24 to 2025-07-25 (same time: 10:30:00)
No conflict with: Off Hours
No conflict with: Agentic AI Project Status Update
No conflict with: Off Hours
No conflict with: Off Hours
No conflict with: Agentic AI Project Status Update
No conflict with: Off Hours
No conflict with: Off Hours
No conflict with: Ag

14.139.128.62 - - [20/Jul/2025 07:44:10] "POST /receive HTTP/1.1" 200 -





 Sending:
 {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-24T10:30:00+05:30",
          "EndTime": "2025-07-24T11:00:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "usertwo.amd@gmail.com",
            "userone.amd@gmail.com",
            "userthree.amd@gmail.com"
          ],
          "Summary": "Agentic AI Project Status Update"
        },
        {
          "StartTime": "2025-07-25T10:30:00+05:30",
          "EndTime": "2025-07-25T11:00:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "userone.amd@gmail.com",
            "usertwo.amd@gmail.com",
            "userthree.amd@gmail.com"
          ],
          "Summary": "CEO position status Update"
        }
      ]
    },
    {
      "email"